In [1]:
import numpy as np
import pandas as pd

import os
pd.set_option('display.max_rows', 10)

In [2]:
def GetHandType(hand:str)->list:
    handType = []
    for character in hand:
        handType.append(hand.count(character))
        hand.replace(character,'')
    handType.sort()
    return handType

In [3]:
def GetHandTypeRank(HandType:list)->int:
    AllTypes = {tuple([5,5,5,5,5]):7,tuple([1,4,4,4,4]):6,tuple([2,2,3,3,3]):5,tuple([1,1,3,3,3]):4,tuple([1,2,2,2,2]):3,
                tuple([1,1,1,2,2]):2,tuple([1,1,1,1,1]):1}
    HandTypeRank = AllTypes[tuple(HandType)]
    return HandTypeRank

In [4]:
def GetIndexRank(hand:str,
                 index:int)->int:
    card = hand[index]
    ranks = {'2':1,'3':2,'4':3,'5':4,'6':5,'7':6,'8':7,'9':8,'T':9,'J':10,'Q':11,'K':12,'A':13}
    rank = ranks[card]
    return rank

In [5]:
data_path = os.path.join('Data','input.txt')
data = pd.read_table(data_path)
data = data.T.reset_index().T.reset_index(drop = True)
data.rename(columns={0:'inputs'},inplace = True)
data['hands'] = data['inputs'].apply(lambda inputs: inputs.split(' ')[0])
data['bids'] = data['inputs'].apply(lambda inputs: int(inputs.split(' ')[1]))
data['handTypes'] = data['hands'].apply(lambda hand: GetHandType(hand))
data['handTypeRanks'] = data['handTypes'].apply(lambda hand: GetHandTypeRank(hand))
for i in range(5):
    data['rank_index'+str(i)] = data['hands'].apply(lambda hand: GetIndexRank(hand,i))
data.sort_values(['handTypeRanks', 'rank_index0','rank_index1','rank_index2','rank_index3','rank_index4'], ascending = True, inplace = True)
data.reset_index(inplace=True)
data['rank'] = data.index+1
data['winnings'] = data['bids']*data['rank']
data

,index,inputs,hands,bids,handTypes,handTypeRanks,rank_index0,rank_index1,rank_index2,rank_index3,rank_index4,rank,winnings
0,890,23QAK 174,23QAK,174,"[1, 1, 1, 1, 1]",1,1,2,11,13,12,1,174
1,831,23K59 604,23K59,604,"[1, 1, 1, 1, 1]",1,1,2,12,4,8,2,1208
2,177,253A7 500,253A7,500,"[1, 1, 1, 1, 1]",1,1,4,2,13,6,3,1500
3,626,2574A 580,2574A,580,"[1, 1, 1, 1, 1]",1,1,4,6,3,13,4,2320
4,883,25J86 410,25J86,410,"[1, 1, 1, 1, 1]",1,1,4,10,7,5,5,2050
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,299,AKAAA 879,AKAAA,879,"[1, 4, 4, 4, 4]",6,13,12,13,13,13,996,875484
996,956,AAA9A 399,AAA9A,399,"[1, 4, 4, 4, 4]",6,13,13,13,8,13,997,397803
997,720,AAAA5 75,AAAA5,75,"[1, 4, 4, 4, 4]",6,13,13,13,13,4,998,74850
998,329,AAAA7 357,AAAA7,357,"[1, 4, 4, 4, 4]",6,13,13,13,13,6,999,356643


In [6]:
data['winnings'].sum()

241344943

In [7]:
def GetIndexRank_2(hand:str,
                   index:int)->int:
    card = hand[index]
    ranks = {'J':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'T':10,'Q':11,'K':12,'A':13}
    rank = ranks[card]
    return rank

In [8]:
def JokerWild(hand:str)->int:
    HandType = GetHandType(hand)
    HandTypeRank = int(GetHandTypeRank(HandType))
    if 'J' in hand:
        for character in ['2','3','4','5','6','7','8','9','T','Q','K','A']:
            newHand = hand.replace('J',character)
            NewHandType = GetHandType(newHand)
            NewHandTypeRank = int(GetHandTypeRank(NewHandType))
            if NewHandTypeRank > HandTypeRank:
                HandTypeRank = NewHandTypeRank
    return HandTypeRank

In [9]:
data_path = os.path.join('Data','input.txt')
data = pd.read_table(data_path)
data = data.T.reset_index().T.reset_index(drop = True)
data.rename(columns={0:'inputs'},inplace = True)
data['hands'] = data['inputs'].apply(lambda inputs: inputs.split(' ')[0])
data['bids'] = data['inputs'].apply(lambda inputs: int(inputs.split(' ')[1]))
data['newHandTypeRanks'] = data['hands'].apply(lambda hand: JokerWild(hand))
for i in range(5):
    data['rank_index'+str(i)] = data['hands'].apply(lambda hand: GetIndexRank_2(hand,i))
data.sort_values(['newHandTypeRanks', 'rank_index0','rank_index1','rank_index2','rank_index3','rank_index4'], ascending = True, inplace = True)
data.reset_index(inplace=True)
data['rank'] = data.index+1
data['winnings'] = data['bids']*data['rank']
data


,index,inputs,hands,bids,newHandTypeRanks,rank_index0,rank_index1,rank_index2,rank_index3,rank_index4,rank,winnings
0,890,23QAK 174,23QAK,174,1,2,3,11,13,12,1,174
1,831,23K59 604,23K59,604,1,2,3,12,5,9,2,1208
2,177,253A7 500,253A7,500,1,2,5,3,13,7,3,1500
3,626,2574A 580,2574A,580,1,2,5,7,4,13,4,2320
4,28,2643K 989,2643K,989,1,2,6,4,3,12,5,4945
...,...,...,...,...,...,...,...,...,...,...,...,...
995,387,TTJTT 341,TTJTT,341,7,10,10,1,10,10,996,339636
996,635,QQQQJ 105,QQQQJ,105,7,11,11,11,11,1,997,104685
997,231,KJKKK 340,KJKKK,340,7,12,1,12,12,12,998,339320
998,753,AJAJA 670,AJAJA,670,7,13,1,13,1,13,999,669330


In [10]:
data['winnings'].sum()

243101568

In [12]:
import sys
sys.path.append(os.pardir)
from src.HelperFunctions import convert_to_py

convert_to_py('7')

[NbConvertApp] Converting notebook 7.ipynb to script
[NbConvertApp] Writing 3458 bytes to 7_raw.py
